In [1]:
import matplotlib
matplotlib.use("nbagg")

import matplotlib.pyplot as plt
import numpy as np

In [2]:
import dlc_bci as bci
from utility import * 

## Get Data
Fetches the data from the server and saves it to a folder at the root 

In [3]:
train_input, train_target, test_input, test_target = import100HzData()
train_input_1000, train_target_1000, test_input_1000, test_target_1000 = import1000HzData()

<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])
<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])


In [4]:
print(train_input.shape) 
print(train_target.shape) 
print(test_input.shape) 
print(test_target.shape) 

torch.Size([316, 28, 50])
torch.Size([316])
torch.Size([100, 28, 50])
torch.Size([100])


## Visualize the data

In [5]:
#rawDataForSingleElectrodeVisualization(train_input)

# Preprocessing

Noise removed - Trend Removed - Offset translated - Normalized 

### A "0" Labelled signal - Without Preprocessing

In [6]:
time = np.linspace(0, 500, 50)

rawDataVisualization(train_input, 0, 'Before Preprocessing - Simple plot of all electrodes - 0 labelled signal')
normalizedSingleSignals(train_input, time, 0, True, 'After Preprocessing - Simple plot of all electrodes - 0 Labelled Signal'); 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### A "1" Labelled signal 

In [7]:
time = np.linspace(0, 500, 50)

rawDataVisualization(train_input, 1, 'Before Preprocessing - Simple plot of all electrodes - 1 Labelled Signal')
normalizedSingleSignals(train_input, time, 1, True, 'After Preprocessing - Simple plot of all electrodes - 1 Labelled Signal');

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Visualize some Normalized signals 

In [8]:
time = np.linspace(0, 500, 50)
normalized_input_train = normalizedSignals(train_input, time, True, 'After Preprocessing - Simple plot of all electrodes - Train')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
normalized_input_test = normalizedSignals(test_input, time, True, 'After Preprocessing - Simple plot of all electrodes - Test')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Neural Net

In [10]:
# Copy the neural network from the Neural Networks section before and modify it to take 18-channel signal (instead of 1-channel images as it was defined).

2. Define a Convolution Neural Network

In [20]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, (1,5)) #nn.Conv2d(nbChannels, outputChannels, kxk Square convolution)
        self.pool = nn.MaxPool2d(1, 1)
        self.conv2 = nn.Conv2d(6, 16, (1,5))
        self.dp = nn.Dropout(0.3)
        self.fc1 = nn.Linear(16 * 28 * 42, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2) #2 output channel one for the left direction and one for the rightweb

    def forward(self, x):
        print(x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dp(x)
        print(x.shape)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dp(x)
        print(x.shape)
        x = x.view(-1, 16 * 28 * 42)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

3. Define a Loss function and optimizer
Let’s use a Classification Cross-Entropy loss and SGD with momentum.

In [21]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

4. Train the network
This is when things start to get interesting. We simply have to loop over our data iterator, and feed the inputs to the network and optimize.

In [22]:
for epoch in range(1000):  # loop over the dataset multiple times
    print ("\nEpoch ", epoch)
    running_loss = 0.0
   
    #inputs = torch.from_numpy(normalized_input_train).float()
    inputs = train_input #x_train
    inputs = torch.unsqueeze(inputs,1) # make it float and insert a fake batch dimension
    labels = train_target #y_train

    # wrap them in Variable
    inputs, labels = Variable(inputs), Variable(labels)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
 
    optimizer.step()

    running_loss += loss.data[0]
    print("Running loss", running_loss)
    
    # Validation accuracy
    params = ["acc", "auc", "fmeasure"]


Epoch  0
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.8420165777206421

Epoch  1
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 1.463278889656067

Epoch  2
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.7526034116744995

Epoch  3
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.7237674593925476

Epoch  4
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.7171506881713867

Epoch  5
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.7149713039398193

Epoch  6
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.7277317643165588

Epoch  7
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28

Running loss 0.5783743262290955

Epoch  63
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.5655201077461243

Epoch  64
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.576505720615387

Epoch  65
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.5637102723121643

Epoch  66
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.5549647808074951

Epoch  67
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.578423261642456

Epoch  68
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.5981153845787048

Epoch  69
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.6227879524230957

Epoch  70
torch.Size([316, 1, 28, 50])
torch.Size([3

torch.Size([316, 16, 28, 42])
Running loss 0.39630722999572754

Epoch  126
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.4523129463195801

Epoch  127
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.45748060941696167

Epoch  128
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.4234907329082489

Epoch  129
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.40615230798721313

Epoch  130
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.40047943592071533

Epoch  131
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.40965530276298523

Epoch  132
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.44459837675094604

Epoch  

torch.Size([316, 16, 28, 42])
Running loss 0.32193365693092346

Epoch  188
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.2943609058856964

Epoch  189
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.3203103244304657

Epoch  190
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.33890846371650696

Epoch  191
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.2725540101528168

Epoch  192
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.2989017963409424

Epoch  193
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.26757878065109253

Epoch  194
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.2818455696105957

Epoch  195

torch.Size([316, 16, 28, 42])
Running loss 0.22991354763507843

Epoch  250
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.21107880771160126

Epoch  251
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.19833332300186157

Epoch  252
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.21576160192489624

Epoch  253
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.15480603277683258

Epoch  254
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.24088157713413239

Epoch  255
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.19209115207195282

Epoch  256
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.21767650544643402

Epoch

...

torch.Size([316, 16, 28, 42])
Running loss 0.01239277608692646

Epoch  746
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.024715010076761246

Epoch  747
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.012806045822799206

Epoch  748
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.023730844259262085

Epoch  749
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.015173138119280338

Epoch  750
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.018487541005015373

Epoch  751
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.02474626526236534

Epoch  752
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.02893255278468132



torch.Size([316, 16, 28, 42])
Running loss 0.008355873636901379

Epoch  808
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.00950007326900959

Epoch  809
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.03032524324953556

Epoch  810
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.007067929487675428

Epoch  811
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.021927107125520706

Epoch  812
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.008262459188699722

Epoch  813
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.006446873303502798

Epoch  814
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.006788815837353468


Running loss 0.005150545854121447

Epoch  869
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.009570630267262459

Epoch  870
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.011702576652169228

Epoch  871
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.01804148033261299

Epoch  872
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.023103080689907074

Epoch  873
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.011588712222874165

Epoch  874
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.018701985478401184

Epoch  875
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.01746932975947857

Epoch  876
torch.Size([316, 1

torch.Size([316, 16, 28, 42])
Running loss 0.00975207518786192

Epoch  931
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.007176001090556383

Epoch  932
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.01042659766972065

Epoch  933
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.014161819592118263

Epoch  934
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.011927402578294277

Epoch  935
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.007470245473086834

Epoch  936
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.018147220835089684

Epoch  937
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.008699804544448853


Running loss 0.0048765456303954124

Epoch  992
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.007239992264658213

Epoch  993
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.01025781873613596

Epoch  994
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.0029804727528244257

Epoch  995
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.0071733188815414906

Epoch  996
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.009784194640815258

Epoch  997
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.004816128872334957

Epoch  998
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
Running loss 0.006393240764737129

Epoch  999
torch.Size([31

5. Test the network on the test data
We have trained the network for 2 passes over the training dataset. But we need to check if the network has learnt anything at all.

We will check this by predicting the class label that the neural network outputs, and checking it against the ground-truth. If the prediction is correct, we add the sample to the list of correct predictions.

Okay, first step. Let us display an image from the test set to get familiar.

# Test Dataset - Prediction 

In [23]:
correct = 0
total = 0

#prediction = torch.from_numpy(normalized_input_test).float() #With normalized input 
prediction = test_input #x_train
prediction_squeezed = torch.unsqueeze(prediction,1) # make it float and insert a fake batch dimension
labels = test_target #y_train

# wrap them in Variable
predictionVar, labelsVar = Variable(prediction_squeezed), Variable(labels)

# forward + backward + optimize
#The outputs are energies for the 10 classes. Higher the energy for a class, the more the network thinks
#that the image is of the particular class. So, let’s get the index of the highest energy:
outputsPred = net(predictionVar) 

_, predicted = torch.max(outputsPred.data, 1) #predicted contains the predicted labels 
total += labels.size(0)
correct += (predicted == labels).sum()

print('Accuracy of the network on the test singals: %d %%' % (100 * correct / total))

torch.Size([100, 1, 28, 50])
torch.Size([100, 6, 28, 46])
torch.Size([100, 16, 28, 42])
Accuracy of the network on the test singals: 63 %


In [26]:
outputsPred

Variable containing:
 -2.6640   3.9532
  8.8456  -9.6214
  0.1550  -0.0297
 -3.5357   4.4109
  1.8238  -1.4063
  3.1895  -3.5055
 -0.2383   0.1084
  5.1676  -4.8362
  1.2249  -1.4031
 -2.1012   3.5118
  4.8394  -4.6521
  2.0946  -1.8490
  4.0792  -5.0915
 -0.5621   0.7031
 -2.6522   3.3002
  2.7138  -3.5626
  1.8130  -2.4442
 -1.9529   1.8262
 -3.1384   3.0853
  5.8101  -6.5961
 -5.5044   6.4523
 -4.7041   5.5059
  0.6310  -0.7007
  6.7001  -7.2470
  0.4713  -1.3409
  4.9509  -5.1220
  2.4243  -2.7349
 -2.9802   3.7466
  2.4709  -2.4638
 -1.1212   1.8925
  3.9360  -4.5434
  2.2988  -3.0212
  4.2348  -4.9989
 -4.7697   5.0937
  2.4832  -2.5855
  2.4431  -2.5215
  0.6484   0.1478
  6.3452  -6.8377
  1.6048  -2.3318
  1.5596  -0.6893
  1.4907  -2.1972
  2.7547  -2.8032
  3.5675  -3.5390
 -1.8041   1.6421
 -0.6064  -0.2004
 -1.1267   1.0639
  2.7806  -2.4651
  4.2490  -4.8654
 -1.5965   1.4930
  0.1024  -1.3900
 -8.8602  10.4361
 -1.7037   1.6221
 -0.0571  -0.1686
  2.7081  -2.7519
  0.969